In [1]:
import numpy as np
import pandas as pd

import model

We want to compare $k$ systems (given by every combination of reorder points and order sizes) and select a system with the smallest total cost $\mu_{i1}$. We want $P(CS) \geq P^*$ provided that $\mu_{i2} - \mu_{i1} \geq d^*$, where the minimal CS probability $P^* > \frac{1}{k} $ and the “indifference” amount $d^* > 0$ are both specifi ed by the analyst.

The statistical procedure for solving this problem involves “two-stage” sampling from each of the $k$ systems. In the fi rst stage we make a fi xed number of replications of each system, then use the resulting variance estimates to determine how many more replications from each system are necessary in a second stage of sampling in order to reach a decision.

In the first-stage sampling, we make $n_0 \geq 2$ replications of each of the $k$ systems and define the first-stage sample means and variances
$$
\bar{X}_i^{(1)}\left(n_0\right)=\frac{\sum_{j=1}^{n_0} X_{i j}}{n_0}
$$
and
$$
S_i^2\left(n_0\right)=\frac{\sum_{j=1}^{n_0}\left[X_{i j}-\bar{X}_i^{(1)}\left(n_0\right)\right]^2}{n_0-1}
$$
for $i=1,2, \ldots, k$. 

In [2]:
reorder_points = [10,20]
order_sizes = [50,60,80,100]
n_0 = 20

res_stage1 = pd.DataFrame(model.run_experiments(reorder_points, order_sizes, num_rep=n_0))
res = res_stage1.groupby(['reorder_point', 'order_size'])['total_cost'].agg(["mean", "var"]).reset_index()
res

Iteration 100 of 160


,reorder_point,order_size,mean,var
0,10,50,124.975,24.093553
1,10,60,123.675,11.138816
2,10,80,126.995,13.169974
3,10,100,132.960,8.675158
4,20,50,120.345,14.747868
5,20,60,121.900,8.623158
6,20,80,127.350,10.341579
7,20,100,134.490,5.053579


In [12]:
res_stage1

,reorder_point,order_size,total_cost,ordering_cost,holding_cost,shortage_cost
0,10,50,125.5,86.9,14.9,23.7
1,10,50,123.6,86.7,15.0,21.9
2,10,50,119.6,83.6,16.0,20.0
3,10,50,126.6,86.7,14.7,25.2
4,10,50,135.7,89.9,14.5,31.3
...,...,...,...,...,...,...
155,20,100,130.5,81.0,47.0,2.5
156,20,100,137.7,84.2,45.8,7.6
157,20,100,133.2,82.0,47.5,3.6
158,20,100,136.2,83.6,47.4,5.2


In [3]:
d_star = 1.0
P_star = 0.95
h2 = 2.267

In [13]:
res['N_i'] = res.apply(lambda row: max(np.ceil((h2**2)*row["var"]/d_star**2), n_0 + 1), axis=1).astype(int)

display(res)

,reorder_point,order_size,mean,var,N_i,mean(N_i - n_0)
0,10,50,124.975,24.093553,124,123.592308
1,10,60,123.675,11.138816,58,124.176316
2,10,80,126.995,13.169974,68,127.143750
3,10,100,132.960,8.675158,45,133.300000
4,20,50,120.345,14.747868,76,119.694643
5,20,60,121.900,8.623158,45,121.608000
6,20,80,127.350,10.341579,54,127.502941
7,20,100,134.490,5.053579,26,133.733333


Next, we make $N_i-n_0$ more replications of system $i(i=1,2, \ldots, k)$ and obtain the second-stage sample means
$$
\bar{X}_i^{(2)}\left(N_i-n_0\right)=\frac{\sum_{j=n_0+1}^{N_i} X_{i j}}{N_i-n_0}
$$

In [5]:
res_stage2 = []

for i in range(res.shape[0]):
    df = pd.DataFrame(model.run_experiments([res['reorder_point'][i]], [res['order_size'][i]], res['N_i'][i]-n_0))
    mean = df['total_cost'].mean()
    res_stage2.append(mean)
    
# print(res_stage2)

Iteration 100 of 104


In [15]:
res_stage2

[123.5923076923077,
 124.1763157894737,
 127.14375,
 133.3,
 119.69464285714285,
 121.60799999999999,
 127.50294117647057,
 133.73333333333332]

In [6]:
res['mean(N_i - n_0)'] = res_stage2
display(res)

,reorder_point,order_size,mean,var,N_i,mean(N_i - n_0)
0,10,50,124.975,24.093553,124,123.592308
1,10,60,123.675,11.138816,58,124.176316
2,10,80,126.995,13.169974,68,127.143750
3,10,100,132.960,8.675158,45,133.300000
4,20,50,120.345,14.747868,76,119.694643
5,20,60,121.900,8.623158,45,121.608000
6,20,80,127.350,10.341579,54,127.502941
7,20,100,134.490,5.053579,26,133.733333


Then define the weights
$$
W_{i 1}=\frac{n_0}{N_i}\left\{1+\sqrt{1-\frac{N_i}{n_0}\left[1-\frac{\left(N_i-n_0\right)\left(d^*\right)^2}{h_1^2 S_i^2\left(n_0\right)}\right]}\right\}
$$
and $W_{i 2}=1-W_{i 1}$, for $i=1,2, \ldots, k$. Finally, define the weighted sample means
$$
\tilde{X}_i\left(N_i\right)=W_{i 1} \bar{X}_i^{(1)}\left(n_0\right)+W_{i 2} \bar{X}_i^{(2)}\left(N_i-n_0\right)
$$
and select the system with the smallest $\tilde{X}_i\left(N_i\right)$.

We implement the same methods for selecting a subset of the total funs, rather than the optimum.

Then we compute the total sample size $N_i$ needed for system $i$ as
$$
N_i=\max \left\{n_0+1,\left\lceil\frac{h_2^2 S_i^2\left(n_0\right)}{\left(d^*\right)^2}\right\rceil\right\}
$$
where $\lceil x\rceil$ is the smallest integer that is greater than or equal to the real number $x$, and $h_2$ (which depends on $k, P^*$, and $n_0$ ) is a constant that can be obtained from a table.

$h_2$ in subset selection differs from $h_1$ in direct selection because it is lower, thus allowing more room for error.

In [7]:
k = res.shape[0]
m = 3

resm = res.copy()
resm['N_i'] = resm.apply(lambda row: max(np.ceil((h2**2)*row["var"]/d_star**2), n_0 + 1), axis=1).astype(int)

In [8]:
resm

,reorder_point,order_size,mean,var,N_i,mean(N_i - n_0)
0,10,50,124.975,24.093553,124,123.592308
1,10,60,123.675,11.138816,58,124.176316
2,10,80,126.995,13.169974,68,127.143750
3,10,100,132.960,8.675158,45,133.300000
4,20,50,120.345,14.747868,76,119.694643
5,20,60,121.900,8.623158,45,121.608000
6,20,80,127.350,10.341579,54,127.502941
7,20,100,134.490,5.053579,26,133.733333


In [9]:
resm['square_brac'] = 1-(resm['N_i']-n_0)*d_star**2/(h2**2*resm['var'])
resm['square_root_arg'] = 1 - resm['N_i']/n_0*resm['square_brac'] 
resm['square_root'] = np.sqrt(resm['square_root_arg'])
resm['W_i1'] = (n_0/resm['N_i'])*(1+resm['square_root'])
resm['W_i2'] = 1-resm['W_i1']
resm['X_tilda'] = resm['W_i1']*resm['mean'] + resm['W_i2']*resm['mean(N_i - n_0)']
resm = resm.drop(['square_brac', 'square_root_arg','square_root'], axis=1)
display(resm)

,reorder_point,order_size,mean,var,N_i,mean(N_i - n_0),W_i1,W_i2,X_tilda
0,10,50,124.975,24.093553,124,123.592308,0.175167,0.824833,123.834510
1,10,60,123.675,11.138816,58,124.176316,0.399392,0.600608,123.976094
2,10,80,126.995,13.169974,68,127.143750,0.325236,0.674764,127.095371
3,10,100,132.960,8.675158,45,133.300000,0.492435,0.507565,133.132572
4,20,50,120.345,14.747868,76,119.694643,0.286139,0.713861,119.880736
5,20,60,121.900,8.623158,45,121.608000,0.506137,0.493863,121.755792
6,20,80,127.350,10.341579,54,127.502941,0.431499,0.568501,127.436947
7,20,100,134.490,5.053579,26,133.733333,0.783113,0.216887,134.325889


In [10]:
resm_stage2 = []

for i in range(resm.shape[0]):
    df = pd.DataFrame(model.run_experiments([resm['reorder_point'][i]], [resm['order_size'][i]], resm['N_i'][i]-n_0))
    mean = df['total_cost'].mean()
    resm_stage2.append(mean)
    
print(resm['N_i']-n_0)


Iteration 100 of 104
0    104
1     38
2     48
3     25
4     56
5     25
6     34
7      6
Name: N_i, dtype: int32


In [11]:
resm['mean(N_i - n_0)'] = resm_stage2

resm['square_brac'] = 1-(resm['N_i']-n_0)*d_star**2/(h2**2*resm['var'])
resm['square_root_arg'] = resm.apply(lambda i: max(0, 1 - i['N_i']/n_0*i['square_brac']), axis=1)

resm['square_root'] = np.sqrt(resm['square_root_arg'])
resm['W_i1'] = (n_0/resm['N_i'])*(1+resm['square_root'])
resm['W_i2'] = 1-resm['W_i1']
resm['X_tilda'] = resm['W_i1']*resm['mean'] + resm['W_i2']*resm['mean(N_i - n_0)']
resm = resm.drop(['square_brac', 'square_root_arg','square_root'], axis=1)

display(resm)




,reorder_point,order_size,mean,var,N_i,mean(N_i - n_0),W_i1,W_i2,X_tilda
0,10,50,124.975,24.093553,124,123.911538,0.175167,0.824833,124.097822
1,10,60,123.675,11.138816,58,122.897368,0.399392,0.600608,123.207948
2,10,80,126.995,13.169974,68,127.970833,0.325236,0.674764,127.653457
3,10,100,132.960,8.675158,45,130.756000,0.492435,0.507565,131.841326
4,20,50,120.345,14.747868,76,119.621429,0.286139,0.713861,119.828471
5,20,60,121.900,8.623158,45,120.892000,0.506137,0.493863,121.402186
6,20,80,127.350,10.341579,54,127.255882,0.431499,0.568501,127.296494
7,20,100,134.490,5.053579,26,133.383333,0.783113,0.216887,134.249979


In [17]:
print(resm.to_latex(index=True,
                  formatters={"name": str.upper},
                  float_format="{:.3f}".format))

\begin{tabular}{lrrrrrrrrr}
\toprule
 & reorder_point & order_size & mean & var & N_i & mean(N_i - n_0) & W_i1 & W_i2 & X_tilda \\
\midrule
0 & 10 & 50 & 124.975 & 24.094 & 124 & 123.912 & 0.175 & 0.825 & 124.098 \\
1 & 10 & 60 & 123.675 & 11.139 & 58 & 122.897 & 0.399 & 0.601 & 123.208 \\
2 & 10 & 80 & 126.995 & 13.170 & 68 & 127.971 & 0.325 & 0.675 & 127.653 \\
3 & 10 & 100 & 132.960 & 8.675 & 45 & 130.756 & 0.492 & 0.508 & 131.841 \\
4 & 20 & 50 & 120.345 & 14.748 & 76 & 119.621 & 0.286 & 0.714 & 119.828 \\
5 & 20 & 60 & 121.900 & 8.623 & 45 & 120.892 & 0.506 & 0.494 & 121.402 \\
6 & 20 & 80 & 127.350 & 10.342 & 54 & 127.256 & 0.431 & 0.569 & 127.296 \\
7 & 20 & 100 & 134.490 & 5.054 & 26 & 133.383 & 0.783 & 0.217 & 134.250 \\
\bottomrule
\end{tabular}



### Second Round - finding optimal

In [35]:
res2 = resm[['reorder_point', 'order_size', 'mean', 'var']].copy()

In [36]:
res2 = res2.iloc[[1,4,5]]

In [37]:
h1 = 2.872
res2['N_i'] = resm.apply(lambda row: max(np.ceil((h1**2)*row["var"]/d_star**2), n_0 + 1), axis=1).astype(int)

In [42]:
resm_stage22 = []

for i in [1,4,5]:
    df = pd.DataFrame(model.run_experiments([res2['reorder_point'][i]], [res2['order_size'][i]], res2['N_i'][i]-n_0))
    mean = df['total_cost'].mean()
    resm_stage22.append(mean)
    


Iteration 100 of 102
1     72
4    102
5     52
Name: N_i, dtype: int32


In [45]:
res2['mean(N_i - n_0)'] = resm_stage22

res2['square_brac'] = 1-(res2['N_i']-n_0)*d_star**2/(h2**2*res2['var'])
res2['square_root_arg'] = 1 - res2['N_i']/n_0*res2['square_brac'] 
res2['square_root'] = np.sqrt(res2['square_root_arg'])
res2['W_i1'] = round((n_0/res2['N_i'])*(1+res2['square_root']),3)
res2['W_i2'] = round(1-res2['W_i1'], 3)
res2['X_tilda'] = round((res2['W_i1']*res2['mean'] + res2['W_i2']*res2['mean(N_i - n_0)']),2)
res2 = res2.drop(['square_brac', 'square_root_arg','square_root'], axis=1)
display(res2)

,reorder_point,order_size,mean,var,N_i,mean(N_i - n_0),W_i1,W_i2,X_tilda
1,10,60,123.675,11.138816,92,124.252778,0.539,0.461,123.94
4,20,50,120.345,14.747868,122,119.318627,0.453,0.547,119.78
5,20,60,121.900,8.623158,72,120.986538,0.632,0.368,121.56


In [47]:
print(res2.to_latex(index=True,
                  formatters={"name": str.upper},
                  float_format="{:.3f}".format))

\begin{tabular}{lrrrrrrrrr}
\toprule
 & reorder_point & order_size & mean & var & N_i & mean(N_i - n_0) & W_i1 & W_i2 & X_tilda \\
\midrule
1 & 10 & 60 & 123.675 & 11.139 & 92 & 124.253 & 0.539 & 0.461 & 123.940 \\
4 & 20 & 50 & 120.345 & 14.748 & 122 & 119.319 & 0.453 & 0.547 & 119.780 \\
5 & 20 & 60 & 121.900 & 8.623 & 72 & 120.987 & 0.632 & 0.368 & 121.560 \\
\bottomrule
\end{tabular}



In [51]:
sum(res2['N_i']) + sum(resm['N_i'])

782